# OpenAI Function Calling

The newer OpenAI Function Calling Functionality allows to to define functions which will be passed to the LLM. The LLM will identify the correct function for the request and provide parameters for the function call.



In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
import openai

def chat(query):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": query}],
    )
    message = response.choices[0].message.content
    return message

In [3]:
query = "How much does pizza salami cost?"
message = chat(query)
message

'The cost of pizza salami can vary depending on the size of the pizza, the restaurant or store it is purchased from, and the region. On average, a medium-sized pizza with salami toppings can cost around $10 to $15.'

To make use of Function calling you need:

1. A function
2. A dictionary which describes the function

In [4]:
import json

def get_pizza_info(pizza_name: str):
    pizza_info = {
        "name": pizza_name,
        "price": "10.99",
    }
    return json.dumps(pizza_info)

In [5]:
functions = [
    {
        "name": "get_pizza_info",
        "description": "Get name and price of a pizza of the restaurant",
        "parameters": {
            "type": "object",
            "properties": {
                "pizza_name": {
                    "type": "string",
                    "description": "The name of the pizza, e.g. Salami",
                },
            },
            "required": ["pizza_name"],
        },
    }
]

In [6]:
def chat(query):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": query}],
        functions=functions, # this is new
    )
    message = response.choices[0].message
    return message

In [7]:
chat("What is the capital of france?")


ChatCompletionMessage(content='The capital of France is Paris.', role='assistant', function_call=None, tool_calls=None)

In [8]:
query = "How much does pizza salami cost?"
message = chat(query)
print(message)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"pizza_name":"Salami"}', name='get_pizza_info'), tool_calls=None)


In [9]:
if message.function_call:
    pizza_name = json.loads(message.function_call.arguments).get("pizza_name")
    print(pizza_name)
    function_response = get_pizza_info(
        pizza_name=pizza_name
    )
    print(function_response)

Salami
{"name": "Salami", "price": "10.99"}


In [10]:
second_response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": query},
        message,
        {
            "role": "function",
            "name": "get_pizza_info",
            "content": function_response,
        },
    ],
)
second_response

ChatCompletion(id='chatcmpl-9BHDTYvmThdpekNN3Mvl3TU5xL7Tq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The cost of a pizza salami is $10.99.', role='assistant', function_call=None, tool_calls=None))], created=1712474815, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=13, prompt_tokens=55, total_tokens=68))